In [3]:
%%writefile goose.py

import os

os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

import tensorflow as tf
import numpy as np
import base64
import json
from kaggle_environments.envs.hungry_geese.hungry_geese import Action
from kaggle_environments import make

NUM_GRID = (7, 11)
NUM_CHANNEL = 6
NUM_ACT = 4

STOCK_X = tf.convert_to_tensor(np.zeros((*NUM_GRID, NUM_CHANNEL)), dtype='float32')
STOCK_ACT = [Action(i + 1) for i in range(NUM_ACT)]

class Block(tf.keras.layers.Layer):
    def __init__(self, flt, **kwargs):
        super(Block, self).__init__(**kwargs)
        
        self.dense = tf.keras.layers.Dense(flt, use_bias=False)
        self.bn = tf.keras.layers.BatchNormalization()
        
    def call(self, inp, training=False):
        x = self.dense(inp)
        x = self.bn(x, training)
        x = tf.nn.relu(x)
        
        return x

class Net(tf.keras.Model):
    def __init__(self, layers, out, stock):
        super(Net, self).__init__()

        self.stats = { 'layers': layers, 'out': out, 'stock': stock }
        self.stock = stock

        self.tower = []

        for l in layers:
            self.tower.append(Block(l))
        
        self.flt = tf.keras.layers.Flatten()
        self.out = tf.keras.layers.Dense(out)
    
    def call(self, inp, training=False):
        x = inp

        if len(x.shape) < 4:
            x = tf.expand_dims(x, 0)
            
        x = self.flt(x)

        for block in self.tower:
            x = block(x, training=training)
            
        x = self.out(x)
            
        return x
    
def act_to_id(act):
    if act.name == 'NORTH':
        return 0
    elif act.name == 'EAST':
        return 1
    elif act.name == 'SOUTH':
        return 2
    elif act.name == 'WEST':
        return 3

def pos_to_coord(pos):
    return pos // NUM_GRID[1], pos % NUM_GRID[1]

def obs_to_x(obs, acts):
    x = [[[0 for _ in range(NUM_CHANNEL)] for _ in range(NUM_GRID[1])] for _ in range(NUM_GRID[0])]
    index = obs['index']
    foods = obs['food']
    geese = obs['geese']

    rc, cc = pos_to_coord(geese[index][0])
    rc = round((NUM_GRID[0] - 1) / 2) - rc
    cc = round((NUM_GRID[1] - 1) / 2) - cc
    
    for food in foods:
        r, c = pos_to_coord(food)
        
        r = (r + rc) % NUM_GRID[0]
        c = (c + cc) % NUM_GRID[1]

        x[r][c][5] = 1

    for i, goose in enumerate(geese):
        if goose:
            r, c = pos_to_coord(goose[0])
            
            r = (r + rc) % NUM_GRID[0]
            c = (c + cc) % NUM_GRID[1]
            
            x[r][c][4] = 1
            
            if acts[i] != None:
                x[r][c][act_to_id(acts[i])] = -1
            
            for ii, block in enumerate(goose):
                if ii == 0:
                    continue
                
                r, c = pos_to_coord(block)
                rt, ct = pos_to_coord(goose[ii - 1])
                
                d = None
                
                if (r - 1) % NUM_GRID[0] == rt:
                    d = 0
                elif (c + 1) % NUM_GRID[1] == ct:
                    d = 1
                elif (r + 1) % NUM_GRID[0] == rt:
                    d = 2
                elif (c - 1) % NUM_GRID[1] == ct:
                    d = 3
                
                r = (r + rc) % NUM_GRID[0]
                c = (c + cc) % NUM_GRID[1]

                x[r][c][4] = 1
                x[r][c][d] = 1
    
    return tf.convert_to_tensor(x, dtype='float32')

shapes = [[462, 512], [512], [512], [512, 128], [128], [128], [128, 128], [128], [128], [128, 256], [256], [256], [512], [512], [128], [128], [128], [128], [256], [256], [256, 4], [4]]

weights = ["...", "Xj1TPQM9Cj0="]

weights = [np.frombuffer(base64.decodebytes(s.encode('utf-8')), dtype=np.half).reshape(shapes[i]) for i, s in enumerate(weights)]
weights = [arr.astype(np.float32) for arr in weights]

critic = Net([512, 128, 128, 256], NUM_ACT, STOCK_X)
critic(critic.stock)
critic.set_weights(weights)

heads = None

def agent(obs, _):
    global critic, heads
    
    acts = [None] * len(obs['geese'])
        
    if heads != None:
        for i, head in enumerate(heads):
            if head and obs['geese'][i]:
                r, c = pos_to_coord(head)
                rr, cc = pos_to_coord(obs['geese'][i][0])

                if (r - 1) % NUM_GRID[0] == rr:
                    acts[i] = STOCK_ACT[0]
                elif (c + 1) % NUM_GRID[1] == cc:
                    acts[i] = STOCK_ACT[1]
                elif (r + 1) % NUM_GRID[0] == rr:
                    acts[i] = STOCK_ACT[2]
                elif (c - 1) % NUM_GRID[1] == cc:
                    acts[i] = STOCK_ACT[3]

    heads = list()

    for goose in obs['geese']:
        if goose:
            heads.append(goose[0])
        else:
            heads.append(None)
            
    x = obs_to_x(obs, acts)
    
    scores = [float(score) for score in critic(x)[0]]
    
    i = scores.index(max(scores))
    
    print(scores, obs['index'], i)
    
    return STOCK_ACT[i].name

Overwriting goose.py


In [4]:
from kaggle_environments import make

env = make('hungry_geese', debug=True)
env.run(['goose.py', 'goose.py', 'goose.py', 'goose.py'])
env.render(mode='ipython')

[25.299501419067383, 26.18805694580078, 25.864450454711914, 24.675975799560547] 0 1
[24.980281829833984, 25.09901237487793, 26.221967697143555, 25.312585830688477] 1 2
[29.094465255737305, 26.298423767089844, 26.023271560668945, 27.176759719848633] 2 0
[28.8027400970459, 25.370328903198242, 26.284128189086914, 28.65091323852539] 3 0
[26.36125373840332, 26.841785430908203, 26.656585693359375, -6.1566386222839355] 0 1
[-2.2560975551605225, 24.718399047851562, 26.901369094848633, 26.15089225769043] 1 2
[29.638586044311523, 26.61531639099121, -14.949649810791016, 28.488353729248047] 2 0
[26.997879028320312, 25.865489959716797, -17.34834861755371, 29.702529907226562] 3 3
[27.6319522857666, 27.77428436279297, 27.114694595336914, -6.328447341918945] 0 1
[-5.642755508422852, 26.46971321105957, -62.919273376464844, 27.550609588623047] 1 3
[28.71466827392578, -16.681100845336914, -9.259538650512695, 31.408679962158203] 2 3
[29.048295974731445, 8.385658264160156, 30.835289001464844, 30.0374584197

[-31.641387939453125, 38.408443450927734, 38.70697021484375, 38.39623260498047] 0 2
[40.480613708496094, 39.185794830322266, 37.878536224365234, -24.553611755371094] 1 0
[39.378761291503906, 39.375858306884766, 39.183021545410156, -4.927796840667725] 2 0
[-30.257917404174805, 38.667049407958984, 39.00834655761719, 38.62596130371094] 3 2
[-15.779169082641602, 39.30512619018555, 40.999732971191406, 40.221317291259766] 0 2
[40.984886169433594, 40.11200714111328, -23.1092529296875, 40.89985275268555] 1 0
[-28.897192001342773, 38.674232482910156, -5.770341873168945, 38.99034118652344] 2 3
[-30.932079315185547, 39.2408447265625, 39.513545989990234, 38.977664947509766] 3 2
[-21.835737228393555, 40.317108154296875, 41.74073028564453, 40.68028259277344] 0 2
[38.8660774230957, 40.08927917480469, -7.899559020996094, 39.94483947753906] 1 1
[41.03343963623047, -23.420581817626953, 12.93087387084961, 41.250919342041016] 2 3
[-32.70561599731445, 39.02756118774414, 39.86857986450195, 39.3887939453125]

[43.957645416259766, 43.24991226196289, -35.46879577636719, 40.298065185546875] 3 0
[-11.849031448364258, -25.48434066772461, 41.85489273071289, 41.939884185791016] 0 3
[39.96561813354492, -38.96786117553711, 42.159244537353516, -38.836090087890625] 1 2
[43.36144256591797, -12.07466983795166, 44.653526306152344, 43.685245513916016] 2 2
[45.060794830322266, 45.0487060546875, -51.7526741027832, 27.331653594970703] 3 0
[39.33607482910156, -15.525081634521484, 42.774478912353516, 25.909879684448242] 0 2
[-21.90842056274414, 39.95789337158203, 41.138023376464844, 33.72182083129883] 1 2
[-31.859375, -14.417701721191406, 44.082183837890625, 43.364227294921875] 2 2
[44.08460235595703, 44.832767486572266, -51.4315185546875, 42.477569580078125] 3 1
[-58.26254653930664, 39.560726165771484, 40.46323013305664, 3.8863611221313477] 0 2
[-7.362245559692383, 40.82548522949219, 41.29433059692383, 40.31565856933594] 1 2
[-20.197303771972656, -5.4645466804504395, 42.446685791015625, 43.00315475463867] 2 3

[40.76380157470703, 40.204071044921875, 39.25643539428711, -22.793806076049805] 0 0
[42.4453010559082, -26.981538772583008, 41.354087829589844, 42.26411437988281] 1 0
[-49.59402847290039, 41.94528579711914, 42.05303955078125, 40.420440673828125] 2 2
[42.8604850769043, -31.00489044189453, 43.08850860595703, 41.494651794433594] 3 2
[42.435997009277344, 42.0413703918457, -35.97449493408203, 42.23398208618164] 0 0
[-22.82198143005371, 42.494468688964844, -8.85302734375, 41.98969650268555] 1 1
[-18.155853271484375, 44.77000045776367, 0.1427631378173828, 43.481746673583984] 2 1
[-39.195396423339844, 41.925315856933594, 42.6037483215332, 42.31187057495117] 3 2
[42.42888641357422, 42.699562072753906, -24.13950538635254, 42.133846282958984] 0 1
[-66.5686264038086, 42.90208053588867, -0.4729032516479492, -2.560668468475342] 1 1
[41.15325164794922, 43.386016845703125, 7.800112724304199, -58.4216194152832] 2 1
[-24.69105339050293, 41.90443801879883, 42.679935455322266, 41.728363037109375] 3 2
[43.

[44.960182189941406, 44.123390197753906, 44.256961822509766, -43.10218811035156] 0 0
[44.83815002441406, 0.48562008142471313, -19.768632888793945, 45.15719223022461] 1 3
[42.90167236328125, -53.56035614013672, 44.30491256713867, 42.56184005737305] 2 2
[44.38593292236328, 29.89888572692871, -1.4800705909729004, 43.1607551574707] 0 0
[45.17799758911133, -33.46355438232422, 45.11595916748047, -64.50358581542969] 1 0
[-4.837597370147705, 42.53084945678711, 44.77211380004883, 43.21553421020508] 2 2
[45.50361251831055, 40.753875732421875, -56.7835693359375, 44.64828872680664] 0 0
[30.56223487854004, 42.912696838378906, -36.45452117919922, -26.77435302734375] 1 1
[-14.946304321289062, 46.25664138793945, 45.414588928222656, 43.346282958984375] 2 1
[44.71142578125, 45.237709045410156, -42.41069030761719, 44.71731185913086] 0 1
[43.80680847167969, 2.929366111755371, -7.065986633300781, -39.13392639160156] 1 0
[44.719661712646484, 45.392478942871094, 44.22108459472656, 9.060220718383789] 2 1
[46.

[46.30167007446289, -16.196441650390625, 46.608909606933594, 46.81595230102539] 0 3
[-4.772387504577637, 46.45176315307617, 47.08605194091797, 48.33030319213867] 1 3
[47.201236724853516, 47.14759063720703, 45.952701568603516, -19.143964767456055] 2 0
[44.61918258666992, -44.58495330810547, 45.0646858215332, 44.13282775878906] 0 2
[45.074703216552734, -37.2772216796875, 47.204349517822266, 46.01551055908203] 1 2
[44.72043991088867, 45.60911560058594, -65.3735122680664, 46.4860725402832] 2 3
[-24.622339248657227, 36.94507598876953, 42.47023391723633, 44.80715560913086] 0 3
[-43.196617126464844, 35.98731231689453, 45.539798736572266, 46.755855560302734] 1 3
[44.59353256225586, -26.100502014160156, 6.796931266784668, 43.016502380371094] 2 0
[44.45984649658203, -22.517074584960938, -59.67206954956055, -24.349220275878906] 0 0
[46.431434631347656, -38.84523391723633, 44.834632873535156, 45.83262634277344] 1 0
[44.72481155395508, 0.2542719841003418, -63.12181091308594, 42.245853424072266] 2 0